In [2]:
import nltk
import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle
from collections import Counter

In [3]:
list(range(3, 5, 3))


[3]

# Getting sentence classifications

In [4]:
f = open("LitCovid_sent_classification_formatted.sents", "r")
sec_class_sents = f.read()
f = open("LitCovid_sent_classification_formatted.txt", "r")
sec_class_txt = f.read()

In [5]:
sec_classes = sec_class_sents.split("\n\n\n")
sec_pmids = [t.split(" | ")[0] for t in sec_class_txt.split("\n")]
sections = {}
for i in range(len(sec_pmids)):
    if sec_pmids[i] != "":
        sections[sec_pmids[i]] = [(t.split("||")[0], t.split("||")[1]) for t in sec_classes[i].split("\n")]

In [6]:
split_sections = {}
for i in tqdm(range(len(sec_pmids))):
    if sec_pmids[i] != "":
        sect = sections[sec_pmids[i]]
        temp_sect = []
        for line in sect:
            if len(line[1].split()) > 30:
                text = line[1].split()
#                 print(text)
                split_sentences = [' '.join(text[i:i+30]) for i in range(0, len(text),30)]
#                 print(split_sentences)
                for sent in split_sentences:
                    temp_sect.append((line[0], sent))
            else:
                temp_sect.append(line)
        
        split_sections[sec_pmids[i]] = temp_sect

100%|███████████████████████████████████████████████████████████████████████████| 6470/6470 [00:00<00:00, 12566.83it/s]


In [7]:
import re
t  = "They had a median ( interquartile range [ IQR ] ) age of 47.0 ( 38.0-70.3 ) years , and 16 ( 38.1 % ) were men .".split()
age = "".join(t[t.index("age")+1:t.index("years")])
regex = re.compile('[a-zA-Z]')
regex.sub('', age)

'47.0(38.0-70.3)'

In [8]:
list(re.finditer("age", "ages you know ages"))

[<re.Match object; span=(0, 3), match='age'>,
 <re.Match object; span=(14, 17), match='age'>]

In [84]:
custom_syns = [['COVID-19', "SARS-CoV-2", "SARS-CoV-2 infection", "COVID-19 infection", 
  "coronavirus disease 2019 ( COVID-19 )", 'Covid-19', 'laboratory-confirmed COVID-19', 
  "coronavirus disease 2019", "COVID-19 disease", "coronavirus disease", "SARS-CoV", 
  "COVID-19 positive", "SARS-COV-2", "SARS-CoV-2 infections",  "Covid-19 infection", 
  "COVID-19 infected", "COVID-19-positive", "SARS-CoV-2 infected", "SARS-CoV-2-infected",
  "2019-nCoV", "COVID-19 infections", "COVID 19", "COVID19", "Covid-19 infection",
  "severe acute respiratory syndrome coronavirus 2 ( SARS-CoV-2 ) infection", 
  'coronavirus disease ( COVID-19 )', "CoV infections", "coronavirus", "2019-new coronavirus",
  "SARS-CoV-2 virus", "coronavirus infections"],
 ["severe COVID-19", "severe and critical COVID-19", "severe COVID-19 infection", 
  'severe coronavirus disease 2019 ( COVID-19 )',
  'severe coronavirus disease 2019 ( COVID-19 )', 
  'severe acute respiratory syndrome ( SARS )', 'severe and critical COVID-19',
  'severe acute respiratory syndrome coronavirus 2',
  'critically ill COVID-19', "severe COVID-19 disease"],
 ["severe COVID-19 pneumonia", "severe pneumonia", "COVID-19 pneumonia", "SARS-CoV-2 pneumonia",
  'novel coronavirus pneumonia', "2019-nCoV pneumonia"],
 ["male", "men", "males", "male sex"],
 ["infection", "infected"],
 ["female", "women", "females", "female sex"],
 ["child", "children"],
 ["severe", "severe disease", "severity", "severe cases", "severely ill", "severe illness"],
 ["critical", "critically ill",  "critical illness"],
 ["comorbidity", "comorbidities"],
 ["ICU", "intensive care unit ( ICU )", "intensive care"],
 ["diabetes", "diabetes mellitus", "diabetic"],
 ["elderly", "older age", "older adults", "older"],
 ["hydroxychloroquine", "chloroquine", "hcq"],
 ["dead", "deceased", "mortality", "died", "death"]]

severe_subtags = [
    ["infection", "infections", "cases", "case", "outcomes", "outcome", "disease", "patient", "patients", "group"],
    ["pneumonia", "event", "events", "complications", "complication", "diabetes", "disease"],
    ["fever", "acute", "ards", "acidosis", "hypercoagulability", "inflammatory", "inflammation"],
    ["course", "courses", "stages", "stage"]]

severe_subtags_names = ["COVID-19", "other diseases", "symptoms", "course"]
age_subtags_names = ["median", "other diseases", "symptoms", "course"]
lower_syns = []
for syn in custom_syns:
    lower_syns.append([s.lower() for s in syn])

    
def get_syn(ent, orig_text, start):
    subtag = "None"
    covid_syns = ["covid", "sars-cov", "corona", "ncov", "cov infections"]
    severe_syns = ["severe"]
    if any(word in ent.lower() for word in covid_syns) and any(word in ent.lower() for word in severe_syns):
        ent = "severe covid-19"
    elif any(word in ent.lower() for word in covid_syns) and "pneumonia" in ent.lower():
        ent = "COVID-19 pneumonia"
#     elif any(word in ent.lower() for word in severe_syns):
#         return "severe"
    else:
        for l in lower_syns:
            if ent.lower() in l:
                ent =  l[0]
        
    if ent == "severe":
        for i in range(len(severe_subtags)):
            orig_text_split = orig_text.lower().split(" ")
            if len(orig_text_split) > start+1 and orig_text_split[start+1] in severe_subtags[i]:
                subtag = severe_subtags_names[i]
                
    if ent == "age":
        orig_text_split = orig_text.lower().split()
        orig_text_split = ["age" if text == "ages" else text for text in orig_text_split]
        
        if "child" in orig_text.lower():
            subtag = "<18"
        elif "years of age" in orig_text.lower():
            age_idx = orig_text_split.index("years")
            
            age = orig_text_split[age_idx-3:age_idx]
            if len(age) == 3:
                if age[1] in ["under", "below", "least", "<"] or age[0] in ["younger", "less"]:
                    age = "<" + age[-1]

                elif age[1] == ["over", "above", "most", ">"] or age[0] in ["older", "more"]:
                    age = ">" + age[-1]   

            subtag = "".join(age)
        
        elif "years" in orig_text.lower(): 
            age = orig_text_split[orig_text_split.index("age")+1:orig_text_split.index("years")]
            if "patients" in age and age[age.index("patients")-1].isnumeric():
                age = age[age.index("patients")+1:orig_text_split.index("years")]
                
            age = " ".join(age)
            age = age.replace("to", "-")
            regex = re.compile('[a-zA-Z]')
            subtag = regex.sub('', age)
        else:
            subtag = "concept"
        
        subtag_ = subtag.replace(" ", "")
        if subtag_.count("+-") == 1 and subtag_[subtag_.find("+-")-1].isnumeric():
            nums = [float(x) for x in re.findall(r"\d+(?:\.\d+)?", subtag_)]
            if len(nums) == 2:
                subtag = str(nums[0]-nums[1])+ "-"+ str(nums[0]+nums[1])
        subtag = subtag.replace("[", "(")
        subtag = subtag.replace("]", ")")
    return ent, subtag

In [83]:
re.findall(r"\d+(?:\.\d+)?", "70.39+-8.02)")

['70.39', '8.02']

# Get data

In [85]:

folder = "../LitCovid_Treatment_json/"
dirs = os.listdir(folder)
pmids = []
entities = []

singulars = 0
for file in tqdm(dirs, desc = "dirs"):
# for file in dirs[:3]:
    try:
        if file not in ["exceptionlist.txt", "label2id.pkl", "predict.tf_record"]:

            #get filepath and read in data
            file_path = os.path.join(folder, file)
            temp = pd.read_json(file_path)
            pmid = file[:-5]
            section = split_sections.get(pmid)

    #             if section[0][0] == "TITLE":
    #                 sec_idx = 1
    #             else:
    #                 sec_idx = 0
    #         print(temp)
    #         print(section)
            for i in range(temp.shape[0]):

                #get entities sentence by sentence
                ents_dict = temp.sentences.values[i].get("entities")
                orig_text = temp.sentences.values[i].get("text")

                if i < len(section):
                    temp_sec = section[i][0]
    #                     sec_idx +=1

                else:
                    temp_sec = "ERROR"

                if len(ents_dict.keys()) > 0:
                    temp_ents = [(ents_dict[k]["text"], ents_dict[k]["class"], ents_dict[k]["negation"], ents_dict[k]["start"], orig_text, "None", temp_sec, pmid) for k in ents_dict.keys()]
                    true_ents = []
                    for i in range(len(temp_ents)):
                        ent = temp_ents[i][0]
                        cl = temp_ents[i][1]
                        neg = temp_ents[i][2]
                        start = temp_ents[i][3]
                        subtag = temp_ents[i][5]
    #                     info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                        added = False

                        if len(ent) > 1: #make sure it's not punctuation

                            if len(ent.split()) > 1: #if phrase

                                #check to see if it's in custom set of synonyms
                                new_ent, subtag = get_syn(ent, orig_text, start)
                                info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                                true_ents.append(info_tuple)

                            else: #if single word
                                if ent.lower() not in stop_words: #if not stop word

                                    #check to see if it's in custom set of synonyms
                                    new_ent, subtag = get_syn(ent, orig_text, start)

                                    if new_ent != ent:
                                        info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                                        true_ents.append(info_tuple)

                                    else:
                                        ent = singularize(ent)
                                        new_ent, subtag = get_syn(ent, orig_text, start)
                                        info_tuple = (new_ent, cl, neg, start, orig_text, subtag, temp_sec, pmid)
                                        true_ents.append(info_tuple)       


                    entities += true_ents
    except Exception as e:
        print(e)
        print(pmid)
        print(section)
        print(temp.sentences.values)


dirs:  10%|██████▉                                                                 | 620/6459 [00:04<00:39, 146.00it/s]

58.2-83.8
58.2-83.8


dirs:  11%|████████▎                                                               | 742/6459 [00:04<00:33, 169.46it/s]

34.0-58.0
34.0-58.0


dirs:  15%|██████████▉                                                             | 982/6459 [00:06<00:31, 176.41it/s]

...

47.2-75.6
47.2-75.6


dirs:  98%|█████████████████████████████████████████████████████████████████████▍ | 6321/6459 [00:41<00:00, 175.86it/s]

43.5-80.1
43.5-80.1


dirs: 100%|███████████████████████████████████████████████████████████████████████| 6459/6459 [00:42<00:00, 153.39it/s]

58.0-86.0
58.0-86.0


In [121]:
ent_df = pd.DataFrame(entities, columns=["text", "type", "negation", "start", "sentence", "subtag", "section", "pmid"])
# 
ent_df



,text,type,negation,start,sentence,subtag,section,pmid
0,covid-19,Participant,0,9,"In this review , we provide an update on CoV i...",None,BACKGROUND,31981224
1,relevant,Participant,0,12,"In this review , we provide an update on CoV i...",None,BACKGROUND,31981224
2,host defense against,Outcome,0,17,"In this review , we provide an update on CoV i...",None,BACKGROUND,31981224
3,covid-19,Participant,0,8,This highlights the importance of immune respo...,None,CONCLUSIONS,31981224
4,covid-19,Participant,0,14,"As of January 22 , 2020 , a total of 571 cases...",None,TITLE,31996494
5,inflammation,Outcome,0,9,"Here , we propose for the first time that infl...",None,RESULTS,32013309
6,covid-19,Participant,0,11,"Here , we propose for the first time that infl...",None,RESULTS,32013309
7,inhibited,modifier,0,13,"Here , we propose for the first time that infl...",None,RESULTS,32013309
8,accelerate,modifier,0,0,accelerate its control .,None,BACKGROUND,32015560
9,control,Outcome,0,2,accelerate its control .,None,BACKGROUND,32015560


In [122]:
def get_range_classification(subtag):
    try:
        subtag_ = ""
        nums = [float(x) for x in re.findall(r"\d+(?:\.\d+)?", subtag)] 
        if nums[0] < 21:
            subtag_ += ",children"
        elif nums[0] < 65:
            subtag_ += ",adult"
        else:
            subtag_ += ",elderly"

        if nums[1] > 65 and subtag != ",elderly":
            subtag_ += ",elderly"

        subtag = subtag_[1:]
        
    except Exception as e:
        subtag = "None"
        print(e)
        print(subtag)
    return subtag 


age_pmids = ent_df[(ent_df.type == "Participant") & (ent_df.text == "age") & ~ent_df.subtag.str.contains("concept")].index

for row in age_pmids:
    subtag = ent_df.loc[row, "subtag"]
    
    if subtag is not "concept" or subtag is not "None":
        subtag = subtag.replace(" ", "")
        l = re.findall(r"\b\d+-\d+\b", subtag)
        s = re.findall(r"\b\d+\s-\s\d+\b", subtag)
        nums = [float(x) for x in re.findall(r"\d+(?:\.\d+)?", subtag)]
        if "<" in subtag:
            nums = re.findall(r"\d+(?:\.\d+)?", subtag)
            if len(nums) == 1:
                if eval(nums[0]) < 21:
                    subtag = "children"
                elif eval(nums[0]) < 65:
                    subtag = "adults,children"
        elif ">" in subtag:
            nums = re.findall(r"\d+(?:\.\d+)?", subtag)
            if len(nums) == 1:
                if eval(nums[0]) > 65:
                    subtag = "elderly"
                else:
                    subtag = "adults,elderly"             
        elif "-" in subtag:
            subtag = get_range_classification(subtag)
            
        elif "(" in subtag and ")" in subtag:
            left = subtag.find("(")
            right = subtag.find(")")
            nums = [float(x) for x in re.findall(r"\d+(?:\.\d+)?", subtag[left+1:right])]
            if len(nums) == 2:
                subtag = get_range_classification(str(nums[0])+ "-"+ str(nums[1]))
            elif len(nums) == 1:
                nums = [float(x) for x in re.findall(r"\d+(?:\.\d+)?", subtag)]
                if len(nums) == 2:
                    subtag = get_range_classification(str(nums[0]-nums[1])+ "-"+ str(nums[0]+nums[1]))
        elif len(nums) == 1:
            if nums[0] <21:
                subtag = "children"
            elif nums[0] > 65:
                subtag = "elderly"
            else:
                subtag = "adult"
    print(subtag)
    ent_df.loc[row, "subtag"] = subtag


children
elderly,elderly
adult
adult,elderly
elderly
adult
adult,elderly
adult
adult
adults,elderly
elderly
elderly
adult
adult,elderly
elderly
elderly
children
adult
adult
elderly
children
children
children
adult
adult
adult
((),54.7(13.7)
((),54.7(13.7)
adult
adult
adult
adult,elderly
adults,children
adults,children
elderly
elderly
adult
adult
adult
adult
adult,elderly
adult
elderly,elderly
adult
adult
adult
adult,elderly
adult
adult,elderly
elderly
adult,elderly
adult
elderly
elderly
adult,elderly
elderly
elderly
children
adult
adult
adult
children,elderly
adult
adult
adults,elderly
elderly
adult
adult
elderly
adult
children
elderly
adult,elderly
children
adult,elderly
adult
elderly,elderly
adult
adult
children
adult
children
list index out of range
None
None
adult
adult
adult
elderly
elderly
elderly
children
adult
elderly
(65.56.3
children
children
adults,children
adults,children
elderly
adult
adult,elderly
adult
elderly
elderly,elderly
adult
adult,elderly
elderly
elderly
children


In [141]:
ent_df.to_csv("LitCovid_treatment_ents")

In [14]:
test_age_df = ent_df[(ent_df.type == "measure") & !ent_df.text.str.contains("year")]
                    

In [149]:
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
temp = ent_df[(ent_df.type == "Participant") & (ent_df.text =="age")]
temp[(temp.subtag.str.contains("concept"))&(temp.subtag.str.contains("children")) & (temp.subtag.str.contains("adult")) & (temp.subtag.str.contains("elderly")) ]

,Unnamed: 0,text,type,negation,start,sentence,subtag,section,pmid


In [148]:
temp

,Unnamed: 0,text,type,negation,start,sentence,subtag,section,pmid
147,147,age,Participant,0,12,A total of 14 children ( 6 months to 14 years ...,children,RESULTS,32051073
719,719,age,Participant,0,2,The mean age of the 52 patients was 59 7 ( SD ...,"elderly,elderly",BACKGROUND,32105632
1728,1728,age,Participant,0,8,"Results : Of 201 patients , the median age was...",adult,BACKGROUND,32167524
1850,1850,age,Participant,0,3,Results : The age of the 54 patients was 68.0 ...,"adult,elderly",METHODS,32171190
1956,1956,age,Participant,0,2,"Indeed , age and disease severity may be corre...",concept,BACKGROUND,32173241
2037,2037,age,Participant,0,15,Compared with general COVID-19 patients ( 45.2...,concept,RESULTS,32173725
2123,2123,age,Participant,0,2,The median age of 69 enrolled patients was 42....,elderly,RESULTS,32176772
2590,2590,age,Participant,0,2,The median age was 47 years ( interquartile ra...,adult,CONCLUSIONS,32198776
3111,3111,age,Participant,0,5,The mean ( SD ) age was 58.50 ( 14.66 ) years .,"adult,elderly",BACKGROUND,32219356
3238,3238,age,Participant,0,6,"Results : All 5 patients ( age range , 36-65 y...",adult,METHODS,32219428


In [ ]:
ent_df[ent_df.type == "Participant"].groupby("pmid").text.unique().iloc[:20].to_csv("example_ptags.csv")

In [ ]:
mort_pmids = np.unique(ent_df[(ent_df.text == "in-hospital mortality")&(ent_df.type == "Outcome")].pmid.values)
ent_df[(ent_df.pmid.isin(mort_pmids))&(ent_df.type == "Intervention")]

In [ ]:
ent_df[(ent_df.text == "severe") & (ent_df.subtag == "None")].sentence.values

In [ ]:
file = "32013309.json"
file_path = os.path.join(folder, file)
temp = pd.read_json(file_path)
    
temp.sentences.values

In [ ]:
sections.get("32013309")

# Send counters to pickle file

In [222]:
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
p_count = Counter(ent_df[ent_df.type == "Participant"].text.values)
i_count = Counter(ent_df[ent_df.type == "Intervention"].text.values)
o_count = Counter(ent_df[ent_df.type == "Outcome"].text.values)

counters = [p_count, i_count, o_count]
c_names = ["p_count", "i_count", "o_count"]
for i in range(len(counters)):
    with open("Pickles/" + c_names[i] + '.pickle', 'wb') as file:
        pickle.dump(counters[i], file)
    

In [1]:
ent_df.pmid.unique().shape

NameError: name 'ent_df' is not defined

# Building weighted graph

In [ ]:
from collections import Counter
import itertools as it

# get data and group
ent_df = pd.read_csv("LitCovid_treatment_ents.csv")
g = ent_df.groupby(["pmid", "type"])

#initialize variables
groups = list(g.groups)
types = ["Participant", "Outcome", "Intervention"]
edge_weight = Counter()
nodes = []

for p in tqdm(ent_df.pmid.values):
    temp = []
    for tup in list(it.product([p], types)):
        if tup in groups:
            
            #get pmid entities
            grp = g.get_group(tup)
            
            #format appropriately
            temp_nodes = ["{} ({})".format(t, tup[1][0]) for t in grp.text.values]
            temp.append(temp_nodes)
            nodes += temp_nodes
        else:
            temp.append([])
            
    #get edges
    products = [list(it.product(temp[0], temp[1])), list(it.product(temp[0], temp[2])), list(it.product(temp[1], temp[2]))]
    
    #add edges to adj list
    for prod in products:
        if prod != []:
            edge_weight.update(prod)

 28%|█████████████████▊                                              | 33980/121669 [03:07<12:23, 117.86it/s]

In [ ]:
with open("Pickles/" + 'wb') as file:
    pickle.dump(edge_weight, file)

In [ ]:
with open('edges.pickle', 'rb') as file:
    edge_weight = pickle.load(file)

In [ ]:
import networkx as nx
from pyvis.network import Network
import matplotlib.pyplot as plt

G = nx.Graph()
for item in edge_weight.most_common(200):
    G.add_edge(item[0][0], item[0][1], weight=item[1]*100)



g = Network(height=800, width=800, notebook=True)
g.barnes_hut()
g.from_nx(G)
g.show_buttons(filter_=['physics'])
g.show("ex.html")

